In [1]:
import pandas as pd

df = pd.read_csv('file/df_health.csv')
df.head()

,category02,question,answer
0,예약,이동훈 도수 치료사분께 도수 치료 진행 받고 싶어요. 가장 빠른 일정을 안내해 주시...,현재 예약 가능한 가장 빠른 일정으로는 9월 4일 열한시가 가능합니다.
1,예약,박진우 원장님 진료 예약 가능 일정을 조회해 주세요. 최대한 빠른 일정이 언제인가요?,"가장 빠른 예약 가능 일정으로는 9월 12일 열시, 13일 열한시 가능합니다."
2,약국,1층 약국은 언제 리모델링이 끝나나요? 여기로 오려니 불편하네요.,다음주 월요일부터 정상 영업에 들어갑니다. 양해 부탁드리겠습니다.
3,증상,증상을 지금 다 말해야 돼요? 그냥 요즘 너무 힘들어요.,의사 선생님이 오시면 환자분의 증상에 대한 자세한 상담을 받으실 수 있습니다.
4,수납,결재할게요. 약값 포함해서 얼마인가요?,약은 약국에서 처방 받으신 뒤 약국에서 결제하시면 됩니다. 진료비와 수액 처방을 포...


In [2]:
import torch
torch.cuda.is_available()

True

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModel.from_pretrained("beomi/KcBERT-base")

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()
# # 질문들의 임베딩 벡터 미리 계산
# def compute_question_embeddings(questions):
#     embeddings = []
#     for question in questions:
#         inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             outputs = model(**inputs)
#         # 문장 전체의 임베딩을 평균으로 취함
#         embedding = outputs.last_hidden_state.mean(dim=1)
#         embeddings.append(embedding)
#     return embeddings

# # 질문에 대한 임베딩 계산
# question_embeddings = compute_question_embeddings(questions)

# # 임베딩을 저장할 때 torch.save() 사용 가능
# torch.save(question_embeddings, 'health_question_embeddings.pth')


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch.nn.functional as F

question_embeddings = torch.load('embeddings/health_question_embeddings.pth', weights_only=True)

# 입력 문장을 임베딩으로 변환
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 임베딩 벡터 반환

# 입력 질문과 사전 임베딩 간의 코사인 유사도 계산
def find_most_similar_answer(input_question, question_embeddings, answers):
    input_embedding = get_embedding(input_question)  # 입력 질문 임베딩

    max_similarity = -1
    best_answer = None

    # 각 사전 계산된 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        similarity = F.cosine_similarity(input_embedding, question_embedding).item()

        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 입력 질문 예시
input_question = "정수기 위치가 어디죠?"
answers = df['answer'].tolist()

# 유사도 기반 답변 찾기
answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)
print("유사도:", max_similarity)


챗봇 답변: 위치 정보를 조회하겠습니다. 병원에서 50m 떨어져 있는 곳에 위치해 있습니다.
유사도: 0.8512148857116699


In [6]:
df['category02'].unique()

array(['예약', '약국', '증상', '수납', '진료접수', '증빙서류', '위치'], dtype=object)

In [7]:
df['category02'].value_counts()

category02
진료접수    3981
수납      3974
증빙서류    3957
예약      3946
약국      3938
증상      3906
위치      3829
Name: count, dtype: int64

In [8]:
# 고유한 카테고리 값 추출
unique_categories = df['category02'].unique()
print(unique_categories)

['예약' '약국' '증상' '수납' '진료접수' '증빙서류' '위치']


In [9]:
# 카테고리와 고유 번호 매핑
category_to_label = {category: idx for idx, category in enumerate(unique_categories)}

# 레이블 추가
df['label'] = df['category02'].map(category_to_label)

# 결과 확인
df.head()
df['label'].unique()

array([0, 1, 2, 3, 4, 5, 6], dtype=int64)

In [10]:
df.to_csv('file/health_result_label.csv', index=False)

In [11]:
from sklearn.model_selection import train_test_split
from collections import Counter
# import random
# CUDA 디버깅을 위한 환경 변수 설정
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# 질문과 답변을 결합
df['input'] = df['question'] + " [SEP] " + df['answer']  # [SEP]는 BERT에서 문장을 구분하는 토큰입니다.

# 훈련 데이터와 평가 데이터로 분리
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    df['input'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# 결과 확인
print("Updated train labels:", Counter(train_labels))
print(set(train_labels))
print(set(eval_labels))

Updated train labels: Counter({4: 3211, 3: 3195, 5: 3156, 1: 3144, 0: 3143, 2: 3114, 6: 3061})
{0, 1, 2, 3, 4, 5, 6}
{0, 1, 2, 3, 4, 5, 6}


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")

# 토크나이징
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True)


In [13]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, num_classes):
        self.encodings = encodings
        self.labels = labels

        # 레이블 유효성 검사
        assert all(label >= 0 for label in self.labels), "Negative label found!"
        assert all(label < num_classes for label in self.labels), "Label out of range!"

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# 데이터셋 생성
num_classes = 7  # 모델의 클래스 수
train_dataset = CustomDataset(train_encodings, train_labels, num_classes)
eval_dataset = CustomDataset(eval_encodings, eval_labels, num_classes)


In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader

model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    logging_dir='./logs',
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 모델 훈련
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/4131 [00:00<?, ?it/s]c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 12%|█▏        | 500/4131 [01:32<11:43,  5.16it/s]

{'loss': 0.2571, 'grad_norm': 0.19159629940986633, 'learning_rate': 4.394819656257565e-05, 'epoch': 0.36}


 24%|██▍       | 1000/4131 [03:50<11:43,  4.45it/s]  

{'loss': 0.078, 'grad_norm': 0.48896560072898865, 'learning_rate': 3.78963931251513e-05, 'epoch': 0.73}


                                                     
 33%|███▎      | 1378/4131 [06:09<6:45:33,  8.84s/it]

...

 97%|█████████▋| 4000/4131 [19:48<00:33,  3.89it/s]  

{'loss': 0.0024, 'grad_norm': 0.001838216558098793, 'learning_rate': 1.5855725006051804e-06, 'epoch': 2.9}


                                                   
100%|██████████| 4131/4131 [21:46<00:00,  3.16it/s]

{'eval_loss': 0.0060028089210391045, 'eval_runtime': 23.0979, 'eval_samples_per_second': 238.42, 'eval_steps_per_second': 29.83, 'epoch': 3.0}
{'train_runtime': 1306.7775, 'train_samples_per_second': 50.561, 'train_steps_per_second': 3.161, 'train_loss': 0.04891339566466617, 'epoch': 3.0}


TrainOutput(global_step=4131, training_loss=0.04891339566466617, metrics={'train_runtime': 1306.7775, 'train_samples_per_second': 50.561, 'train_steps_per_second': 3.161, 'total_flos': 2342907678970800.0, 'train_loss': 0.04891339566466617, 'epoch': 3.0})

In [15]:
trainer.evaluate()
# {'eval_loss': 0.01164562813937664,
#  'eval_runtime': 36.5896,
#  'eval_samples_per_second': 188.305,
#  'eval_steps_per_second': 23.559,
#  'epoch': 3.0}

  0%|          | 0/689 [00:00<?, ?it/s]

100%|██████████| 689/689 [00:24<00:00, 28.56it/s]


{'eval_loss': 0.0060028089210391045,
 'eval_runtime': 24.1581,
 'eval_samples_per_second': 227.956,
 'eval_steps_per_second': 28.52,
 'epoch': 3.0}

In [16]:
import os
import torch

# 저장할 폴더 경로 설정
save_dir = 'model'

# 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 저장할 파일 경로 설정
save_path = os.path.join(save_dir, 'health_model_weights_v1.pth')

# 모델의 상태 사전(state_dict) 저장 -> 호환성이 더 좋음
torch.save(model.state_dict(), save_path)

In [17]:
import torch
from transformers import AutoModelForSequenceClassification
# 모델 구조를 정의한 코드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)

# 모델 가중치를 CPU로 로드
model.load_state_dict(torch.load('model/health_model_weights_v1.pth', weights_only=True))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [18]:
import numpy as np

questions = df['question'].tolist()
answers = df['answer'].tolist()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

def get_embeddings(questions, tokenizer, model):
    model.eval()  # 평가 모드로 전환
    embeddings = []

    with torch.no_grad():
        for question in questions:
            # 입력 토큰화 (입력을 GPU로 이동)
            inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)

            # 출력 얻기
            outputs = model(**inputs, output_hidden_states=True)  # hidden_states를 반환하도록 설정
            
            # BERT의 마지막 레이어 hidden_states에서 [CLS] 토큰의 임베딩을 사용
            cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [batch_size, hidden_size]

            # CPU로 이동하고 numpy 배열로 변환하여 임베딩 목록에 추가
            embeddings.append(cls_embedding.squeeze().cpu().numpy())

    return np.array(embeddings)  # 모든 임베딩을 NumPy 배열로 반환


# train 후 새로운 임베딩 생성
question_embeddings = get_embeddings(questions, tokenizer, model)

# 저장할 폴더 경로
save_dir = 'embeddings'

# 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 저장할 파일 경로
save_path = os.path.join(save_dir, 'train_question_embeddings_v1.pth')

# 임베딩 저장
torch.save(question_embeddings, save_path)

In [19]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드하고 GPU로 이동
question_embeddings = torch.load('embeddings/train_question_embeddings_v1.pth')

# 만약 question_embeddings가 numpy 배열로 되어 있다면, 이를 torch.Tensor로 변환
if isinstance(question_embeddings, np.ndarray):
    question_embeddings = [torch.tensor(embedding).to(device) for embedding in question_embeddings]  # numpy 배열을 텐서로 변환

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 평가 모드로 설정
model.eval()

def get_embedding(input_question, tokenizer, model, device):
    # 입력 문장을 토크나이즈하고 GPU/CPU로 이동
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
    
    return cls_embedding.squeeze().to(device)  # 텐서를 GPU로 반환

# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model, device):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model, device)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding이 numpy 배열인 경우 텐서로 변환
        if isinstance(question_embedding, np.ndarray):
            question_embedding = torch.tensor(question_embedding).to(device)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), question_embedding.unsqueeze(0)).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers, df, device):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model, device)
    
    return best_answer, cosine_similarity, predicted_label


c:\Temp\ipykernel_6140\2989334435.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/train_question_embeddings_v1.pth')


In [20]:
# 예시 질문
input_question = "정수기 어디있죠?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(
    input_question, tokenizer, model, question_embeddings, answers, df, device
)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)
# 예측된 레이블: 0
# 최고 유사도 답변: 청력검사 대기실은 왼쪽 복도로 직진하시면 위치해 있습니다.
# 코사인 유사도: 0.9980705380439758

예측된 레이블: 6
최고 유사도 답변: 네 진료실 앞 의자에서 대기해 주시면 됩니다.
코사인 유사도: 0.9959897994995117


In [31]:
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from skopt import space

# 모델 구조 정의
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")

# 성능 평가 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Accuracy
    accuracy = accuracy_score(labels, predictions)

    # F1 Score (macro average for multi-class classification)
    f1 = f1_score(labels, predictions, average="macro")

    # NumPy에서 생성된 값을 기본 Python int로 변환
    return {
        "accuracy": float(accuracy),  # float으로 변환
        "f1": float(f1),              # float으로 변환
    }

# 베이지안 최적화의 목적 함수
def objective(params):
    learning_rate = float(params[0])  # float로 변환
    batch_size = int(params[1])        # int로 변환
    num_train_epochs = int(params[2])  # int로 변환

     # 배치 사이즈가 유효한지 확인
    if batch_size < 1:
        raise ValueError(f"Invalid batch size: {batch_size}. Must be a positive integer.")
    
    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,  # batch_size 사용
        learning_rate=learning_rate,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs',
        logging_steps=10,
        save_total_limit=1,
        load_best_model_at_end=True,
    )

    # Trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    # 모델 학습
    trainer.train()

    # 평가 및 최적화 지표 반환
    eval_result = trainer.evaluate()

    # eval_loss 변환
    eval_loss = eval_result.get("eval_loss", np.nan)
    if isinstance(eval_loss, (np.float32, np.float64, np.int64)):
        eval_loss = float(eval_loss)

    return eval_loss  # 최종적으로 Python float으로 반환


# space 정의
space = [
    space.Real(1e-5, 1e-4, name='learning_rate', prior='log-uniform'),  # float
    space.Integer(8, 32, name='batch_size'),  # int
    space.Integer(2, 5, name='num_train_epochs')  # int
]


# 베이지안 최적화 실행
result = gp_minimize(objective, space, n_calls=10, random_state=0)

# 최적화된 하이퍼파라미터 출력
print(f"Best hyperparameters: Learning Rate = {result.x[0]}, Batch Size = {result.x[1]}, Num Epochs = {result.x[2]}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 10/3935 [00:02<17:34,  3.72it/s]

{'loss': 1.6503, 'grad_norm': 7.324362277984619, 'learning_rate': 3.906065688915182e-05, 'epoch': 0.01}


  1%|          | 20/3935 [00:05<17:26,  3.74it/s]

{'loss': 0.8572, 'grad_norm': 39.257896423339844, 'learning_rate': 3.8961139291982e-05, 'epoch': 0.03}


  1%|          | 30/3935 [00:08<17:34,  3.70it/s]

...

{'eval_loss': 0.012767656706273556, 'eval_accuracy': 0.9987288905029962, 'eval_f1': 0.998730593978032, 'eval_runtime': 24.8996, 'eval_samples_per_second': 221.168, 'eval_steps_per_second': 27.671, 'epoch': 3.0}


100%|██████████| 6009/6009 [26:37<00:00,  3.76it/s]


{'train_runtime': 1597.2577, 'train_samples_per_second': 41.366, 'train_steps_per_second': 3.762, 'train_loss': 0.014041052434696598, 'epoch': 3.0}


100%|██████████| 689/689 [00:24<00:00, 28.62it/s]

Best hyperparameters: Learning Rate = 3.916017448632163e-05, Batch Size = 28, Num Epochs = 5


In [32]:
# 최적화된 하이퍼파라미터 출력
print(f"Best hyperparameters: Learning Rate = {result.x[0]}, Batch Size = {result.x[1]}, Num Epochs = {result.x[2]}")

Best hyperparameters: Learning Rate = 3.916017448632163e-05, Batch Size = 28, Num Epochs = 5


In [33]:
eval_results = trainer.evaluate()
print(eval_results)


#{'eval_loss': 0.020677322521805763,
#  'eval_runtime': 30.2996,
#  'eval_samples_per_second': 227.495,
#  'eval_steps_per_second': 28.449,
#  'epoch': 3.0}
# {'eval_loss': 0.01164562813937664, 'eval_runtime': 32.5923, 'eval_samples_per_second': 211.4, 'eval_steps_per_second': 26.448, 'epoch': 3.0}

100%|██████████| 689/689 [00:26<00:00, 25.69it/s]

{'eval_loss': 0.0060028089210391045, 'eval_runtime': 26.855, 'eval_samples_per_second': 205.064, 'eval_steps_per_second': 25.656, 'epoch': 3.0}


In [34]:
# 학습된 모델과 토크나이저 저장
model.save_pretrained('./bayesian_saved_model')
tokenizer.save_pretrained('./bayesian_saved_model')

('./bayesian_saved_model\\tokenizer_config.json',
 './bayesian_saved_model\\special_tokens_map.json',
 './bayesian_saved_model\\vocab.txt',
 './bayesian_saved_model\\added_tokens.json',
 './bayesian_saved_model\\tokenizer.json')

In [35]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 저장된 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained('./bayesian_saved_model')
tokenizer = AutoTokenizer.from_pretrained('./bayesian_saved_model')

In [36]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [37]:
import numpy as np
import torch
questions = df['question'].tolist()
answers = df['answer'].tolist()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

def get_embeddings(questions, tokenizer, model):
    model.eval()  # 평가 모드로 전환
    embeddings = []

    with torch.no_grad():
        for question in questions:
            # 입력 토큰화 (입력을 GPU로 이동)
            inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)

            # 출력 얻기
            outputs = model(**inputs, output_hidden_states=True)  # hidden_states를 반환하도록 설정
            
            # BERT의 마지막 레이어 hidden_states에서 [CLS] 토큰의 임베딩을 사용
            cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [batch_size, hidden_size]

            # CPU로 이동하고 numpy 배열로 변환하여 임베딩 목록에 추가
            embeddings.append(cls_embedding.squeeze().cpu().numpy())

    return np.array(embeddings)  # 모든 임베딩을 NumPy 배열로 반환


# fine-tuning 후 새로운 임베딩 생성
question_embeddings = get_embeddings(questions, tokenizer, model)

# 저장할 폴더 경로
save_dir = 'embeddings'

# 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 저장할 파일 경로
save_path = os.path.join(save_dir, 'health_BO_question_embeddings.pth')

# 임베딩 저장
torch.save(question_embeddings, save_path)

In [40]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드
question_embeddings = torch.load('embeddings/health_BO_question_embeddings.pth')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 평가 모드로 설정
model.eval()

def get_embedding(input_question, tokenizer, model, device):
    # 입력 문장을 토크나이즈하고 GPU/CPU로 이동
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
    
    return cls_embedding.squeeze().to(device)  # 텐서를 GPU로 반환

# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model, device):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model, device)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding이 numpy 배열인 경우 텐서로 변환
        if isinstance(question_embedding, np.ndarray):
            question_embedding = torch.tensor(question_embedding).to(device)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), question_embedding.unsqueeze(0)).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers, df, device):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model, device)
    
    return best_answer, cosine_similarity, predicted_label

c:\Temp\ipykernel_6140\3854730239.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/health_BO_question_embeddings.pth')


In [42]:
# 예시 질문
input_question = "너무 목이 마른데요, 혹시 정수기 같은 건 없나요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(
    input_question, tokenizer, model, question_embeddings, answers, df, device
)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)



예측된 레이블: 6
최고 유사도 답변: 정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.
코사인 유사도: 1.0


In [44]:
# 예시 질문
input_question = "정수기는 위치가 어디죠?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(
    input_question, tokenizer, model, question_embeddings, answers, df, device
)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)



예측된 레이블: 6
최고 유사도 답변: 저와 함께 이동해 주시면 직접 안내를 도와드릴 수 있습니다.
코사인 유사도: 0.9997997283935547
